In [1]:
!nvidia-smi


Tue Feb 18 16:13:19 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    Off  | 00000000:00:05.0 Off |                  Off |
| 41%   32C    P8    13W / 140W |      1MiB / 16376MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

In [15]:
!pip install bitsandbytes==0.43.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 28.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.40.2
    Uninstalling bitsandbytes-0.40.2:
      Successfully uninstalled bitsandbytes-0.40.2


In [3]:
from huggingface_hub import login
login()

In [1]:

from datasets import load_dataset

dataset_name = 'nisaar/Articles_Constitution_3300_Instruction_Set'
dataset = load_dataset(dataset_name, split="train")

In [2]:

dataset[1]

{'instruction': 'Identify and summarize the key legal issues in the provided case.',
 'input': 'Case Citation: Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278. The case revolves around the termination of employment of the respondents by the appellant without assigning any reason by invoking Rule 9(i) of the service rules. The respondents challenged the termination orders and the validity of Rule 9(i) in the High Court under Article 226. The Division Bench of the High Court struck down Rule 9(i) as arbitrary and violative of Article 14. The appellant filed appeals in the Supreme Court against the High Court judgment.',
 'output': "The key legal issues in the case Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr. are as follows: 1. The first issue is whether a government company falls under the definition of 'State' as per Article 12 of the Indian Constitution. This is important as the constitutional

In [3]:
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output', 'prompt'],
    num_rows: 3311
})


In [4]:

from datasets import Dataset

# Assuming `dataset` is your Dataset object
dataset = dataset.map(lambda example: {'text': example['prompt'] + example['output']})

In [5]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'prompt', 'text'],
    num_rows: 3311
})

In [6]:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

In [7]:

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
     

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

2025-02-18 16:27:44.175145: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-18 16:27:44.211695: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-18 16:27:44.211760: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-18 16:27:44.212976: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-18 16:27:44.220335: I tensorflow/core/platform/cpu_feature_guar

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [10]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

In [11]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [12]:
from transformers import TrainingArguments

In [13]:
output_dir = "./results_fff"
per_device_train_batch_size = 1
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_steps = 1
num_train_epochs = 4
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 20
warmup_ratio = 0.03
lr_scheduler_type = "linear"

In [14]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [15]:
from trl import SFTTrainer

In [16]:
max_seq_length = 2048

In [17]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=training_arguments,
)

/tmp/ipykernel_466/2529246835.py:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Tokenizing train dataset:   0%|          | 0/3311 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3311 [00:00<?, ? examples/s]

[2025-02-18 16:28:37,693] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [18]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [19]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,1.234500
2,1.131500
3,1.223100
4,1.241900
5,1.030100
6,1.128300
7,1.241700
8,1.112500
9,1.011800
10,1.166300


TrainOutput(global_step=20, training_loss=1.1490951359272004, metrics={'train_runtime': 182.9461, 'train_samples_per_second': 0.219, 'train_steps_per_second': 0.109, 'total_flos': 1037135611748352.0, 'train_loss': 1.1490951359272004})

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fd75c20dc90>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fd75c359bd0, execution_count=19 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fd75c60ad90, raw_cell="trainer.train()" store_history=True silent=False shell_futures=True cell_id=d272ec6d-7446-430f-8fd3-b7aadfe9a272> result=TrainOutput(global_step=20, training_loss=1.1490951359272004, metrics={'train_runtime': 182.9461, 'train_samples_per_second': 0.219, 'train_steps_per_second': 0.109, 'total_flos': 1037135611748352.0, 'train_loss': 1.1490951359272004})>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [20]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fd75c20dc90>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7fd753cb9d90, raw_cell="model_to_save = trainer.model.module if hasattr(tr.." store_history=True silent=False shell_futures=True cell_id=10cd485b-6c86-4be9-bc6a-a49aed33c39e>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fd75c20dc90>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fd750921dd0, execution_count=20 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fd753cb9d90, raw_cell="model_to_save = trainer.model.module if hasattr(tr.." store_history=True silent=False shell_futures=True cell_id=10cd485b-6c86-4be9-bc6a-a49aed33c39e> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [22]:
import wandb
wandb.finish()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fd75c20dc90>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7fd75c0cf950, raw_cell="import wandb
wandb.finish()" store_history=True silent=False shell_futures=True cell_id=3bea44e2-201b-4789-8fc8-08386a24fff0>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

train/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇███
train/global_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇███
train/grad_norm,▁▁▂▂▃▃▃▄▅▄▄▅▅▅▆▇▆█▇▆
train/learning_rate,██▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁
train/loss,▅▃▅▅▁▃▅▃▁▄▂▃▁▅█▁▃▄▄▂
train/mean_token_accuracy,▄▅▂▃▅▅▂▆▄▃▇▄█▁▁█▃▃▄▆
total_flos,1037135611748352.0
train/epoch,0.01208
train/global_step,20
train/grad_norm,0.15816
train/learning_rate,0.0


In [23]:

lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'TinyPixel/Llama-2-7B-bf16-sharded' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


In [24]:
dataset['text'][0]

"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnalyze and explain the legal reasoning behind the judgment in the given case.\n\n### Input:\nCentral Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278\n\n### Response:The Supreme Court in this case applied a broad interpretation of the term 'State' under Article 12 of the Constitution. The court reasoned that a government company undertaking public functions qualifies as 'State' based on factors like government control, public importance of activities etc. This interpretation was based on previous decisions that have defined 'State' under Article 12 broadly to include various agencies and instrumentalities beyond just statutory bodies. The court also applied the principle that unreasonable and arbitrary contractual terms can be struck down under Article 14 

In [25]:
dataset['output'][0]

"The Supreme Court in this case applied a broad interpretation of the term 'State' under Article 12 of the Constitution. The court reasoned that a government company undertaking public functions qualifies as 'State' based on factors like government control, public importance of activities etc. This interpretation was based on previous decisions that have defined 'State' under Article 12 broadly to include various agencies and instrumentalities beyond just statutory bodies. The court also applied the principle that unreasonable and arbitrary contractual terms can be struck down under Article 14 of the Constitution. The court found that Rule 9(i) of the service rules, which allowed for termination of service without reason, conferred unfettered power to terminate employment without hearing. This was deemed arbitrary and violative of principles of natural justice and right to equality under Article 14. Furthermore, the court held that the right to life and livelihood under Article 21 is a

In [26]:
text = dataset['text'][0]
device = "cuda:0"
     

In [27]:
inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Analyze and explain the legal reasoning behind the judgment in the given case.

### Input:
Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278

### Response:The Supreme Court in this case applied a broad interpretation of the term 'State' under Article 12 of the Constitution. The court reasoned that a government company undertaking public functions qualifies as 'State' based on factors like government control, public importance of activities etc. This interpretation was based on previous decisions that have defined 'State' under Article 12 broadly to include various agencies and instrumentalities beyond just statutory bodies. The court also applied the principle that unreasonable and arbitrary contractual terms can be struck down under Article 14 of the Co

In [28]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): LlamaForCausalLM(
          (model): LlamaModel(
            (embed_tokens): Embedding(32000, 4096, padding_idx=0)
            (layers): ModuleList(
              (0-31): 32 x LlamaDecoderLayer(
                (self_attn): LlamaAttention(
                  (q_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=4096, out_features=64, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=64, out_features=4096, bias=False)
                    )
                    (lora_embedding_A): ParameterDict(